In [1]:
import gc
import os

import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset

import pytorch_lightning as pl

In [2]:
torch.__version__, torch.cuda.is_available()

('1.13.1+cu117', True)

In [3]:
pl.seed_everything(56)

56

In [4]:
columns = ['variantid', 'main_pic_embeddings_resnet_v1', 'name_bert_64']
train_data = pd.read_parquet('./datasets/train_data.parquet', columns=columns).set_index('variantid')
test_data = pd.read_parquet('./datasets/test_data.parquet', columns=columns).set_index('variantid')
train_data

,main_pic_embeddings_resnet_v1,name_bert_64
variantid,,
51195767,"[[0.04603629, 0.18839523, -0.09973055, -0.6636...","[-0.47045058, 0.67237014, 0.48984158, -0.54485..."
53565809,"[[1.1471839, -0.665361, 0.7745614, 0.26716197,...","[-0.6575592, 0.6522429, 0.5426037, -0.54347897..."
56763357,"[[-0.90570974, 1.0296293, 1.0769907, 0.27746, ...","[-0.7384308, 0.70784587, 0.3012653, -0.3583719..."
56961772,"[[0.13133773, -0.5577079, 0.32498044, 0.191717...","[-0.44812852, 0.5283565, 0.28981736, -0.506841..."
61054740,"[[0.21696381, 0.10989461, -0.08012986, 0.69186...","[-0.72692573, 0.75206333, 0.37740713, -0.52502..."
...,...,...
820128810,"[[-1.4492652, -0.80129164, -0.12344764, 0.7194...","[-0.8253241, 0.6785133, 0.53978086, -0.4888316..."
821135769,"[[0.012127608, -0.8534423, 0.5415518, -0.44912...","[-0.7413257, 0.46105132, 0.5639801, -0.5462132..."
822095690,"[[0.4248176, -0.15944786, -0.22844064, 0.42768...","[-0.49261805, 0.56726897, 0.7037877, -0.697246..."


In [5]:
train_pairs = pd.read_parquet('./datasets/train_pairs_w_target.parquet')
test_pairs = pd.read_parquet('./datasets/test_pairs_wo_target.parquet')
train_pairs['target'] = train_pairs['target'].astype(int)
train_pairs

,target,variantid1,variantid2
0,0,51197862,51198054
1,1,53062686,536165289
2,1,53602615,587809782
3,1,53888651,89598677
4,0,56930698,551526166
...,...,...,...
306535,0,817327230,822083612
306536,0,817560551,818069912
306537,0,817854719,817857267
306538,0,820036017,820037019


In [6]:
name_labse_embs = pd.read_parquet('F:/name_labse_embs.parquet').set_index('variantid')
name_labse_embs

,name_labse_768
variantid,
51195767,"[-0.033874325, 0.03722446, 0.0029757991, 0.068..."
53565809,"[0.015568526, -0.03899538, 0.064447366, 0.0383..."
56763357,"[-0.033072222, -0.04237577, 0.020771954, 0.065..."
56961772,"[0.014727573, -0.025661988, 0.023943473, -0.00..."
61054740,"[0.043145332, -0.052424084, 0.017260496, 0.045..."
...,...
820128810,"[-0.003678058, -0.031628493, 0.0065589263, 0.0..."
821135769,"[-0.06858361, 0.027011767, -0.016400583, -0.02..."
822095690,"[-0.04474233, -0.034224413, 0.026076552, 0.026..."


In [7]:
from sklearn.model_selection import train_test_split

train_pairs, val_pairs = train_test_split(train_pairs, test_size=1/3, random_state=56)

In [8]:
class Args:
    batch_size = 128
    
args = Args()

In [9]:
class ItemsDataset(Dataset):
    def __init__(self, pairs, data):
        super().__init__()
        self.pairs = pairs.values
        self.main_pic_embs = data['main_pic_embeddings_resnet_v1']
        self.name_embs = data['name_bert_64']
        self.name_labse_embs = name_labse_embs['name_labse_768']
        self.pairs_len = len(self.pairs)
        
    def __len__(self):
        return self.pairs_len

    def __getitem__(self, idx):
        target, id1, id2 = self.pairs[idx, :]
        return (
            torch.tensor(np.concatenate([self.main_pic_embs[id1][0], self.name_embs[id1], self.name_labse_embs[id1]])),
            torch.tensor(np.concatenate([self.main_pic_embs[id2][0], self.name_embs[id2], self.name_labse_embs[id2]])),
            target
        )

In [10]:
def get_data_loader(pairs, data, batch_size, drop_last, shuffle):
    dataset = ItemsDataset(pairs, data)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        num_workers=0,
        drop_last=drop_last,
        shuffle=shuffle,
        pin_memory=True
    )
    return data_loader

In [11]:
def get_loaders(args):
    train_loader = get_data_loader(
        pairs=train_pairs,
        data=train_data,
        batch_size=args.batch_size,
        drop_last=True,
        shuffle=True
    )
    
    val_loader = get_data_loader(
        pairs=val_pairs,
        data=train_data,
        batch_size=args.batch_size,
        drop_last=False,
        shuffle=False
    )
    return train_loader, val_loader

In [12]:
train_loader, val_loader = get_loaders(args)
len(train_loader), len(val_loader)

(1596, 799)

In [13]:
for x1, x2, target in train_loader:
    print(x1, x2, target)
    break

tensor([[ 0.1161, -1.0675,  0.3716,  ..., -0.0537, -0.0297,  0.0599],
        [-0.0278,  0.0919,  0.4331,  ..., -0.0140, -0.0102, -0.0098],
        [ 0.2479, -0.3031,  0.0908,  ..., -0.0060, -0.0081,  0.0303],
        ...,
        [-0.2100, -0.5099,  0.3891,  ...,  0.0434, -0.0274, -0.0064],
        [ 0.6944,  0.5330, -1.5235,  ...,  0.0318, -0.0834,  0.0291],
        [ 0.1003, -0.4157, -0.1716,  ...,  0.0218,  0.0101,  0.0634]]) tensor([[ 0.0578, -1.3894,  0.4709,  ..., -0.0489, -0.0226,  0.0526],
        [-0.0278,  0.0919,  0.4331,  ...,  0.0202,  0.0052, -0.0047],
        [ 0.2479, -0.3031,  0.0908,  ...,  0.0039, -0.0087,  0.0255],
        ...,
        [-0.1911, -0.5303,  0.3174,  ...,  0.0382, -0.0152, -0.0030],
        [ 0.7403,  0.3063, -1.2884,  ...,  0.0344, -0.0862,  0.0169],
        [ 0.1471, -0.1464, -0.2291,  ...,  0.0140,  0.0031,  0.0693]]) tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1,
        0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,

In [14]:
from sklearn.metrics import roc_auc_score

class Net(pl.LightningModule):
    margin = 0.75
    
    def __init__(self):
        super().__init__()
        
        # self.linear = nn.Linear(128 + 64, 256)
        
        features_num = 128 + 64 + 768
        # output_size = 256
        # embedding_size = (features_num + output_size) // 2
        embedding_size = 256
        self.neck = nn.Sequential(
            nn.BatchNorm1d(features_num),
            nn.Linear(features_num, embedding_size, bias=False),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(embedding_size),
            nn.Linear(embedding_size, embedding_size, bias=False),
            nn.BatchNorm1d(embedding_size),
        )
        # self.linear = nn.Linear(embedding_size, output_size)
        
    def forward(self, x):
        # return self.linear(x)
        # return self.linear(self.neck(x))
        return self.neck(x)
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(), lr=1e-4, betas=(0.9, 0.999), weight_decay=0.05
        )
        # optimizer = torch.optim.SGD(self.parameters(), lr=1e-1, momentum=0.9, weight_decay=0.0001)
        return (
            {
                "optimizer": optimizer,
            },
        )
    
    def training_step(self, batch, batch_idx):
        self.log('step', batch_idx, logger=True, on_epoch=True)
        x1, x2, labels = batch
        out1 = self.forward(x1)
        out2 = self.forward(x2)
        
        fnorm = torch.norm(out1, p=2, dim=1, keepdim=True)
        out1 = out1.div(fnorm.expand_as(out1))
        fnorm = torch.norm(out2, p=2, dim=1, keepdim=True)
        out2 = out2.div(fnorm.expand_as(out2))
        
        dists = nn.PairwiseDistance()(out1, out2)
        loss = (labels) * torch.pow(dists, 2) + (1 - labels) * torch.pow(torch.clamp(self.margin - dists, min=0.0), 2)
        loss = torch.mean(loss)
        self.log("train_loss", loss, logger=True, on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_idx):        
        x1, x2, labels = batch
        out1 = self.forward(x1)
        out2 = self.forward(x2)
        
        fnorm = torch.norm(out1, p=2, dim=1, keepdim=True)
        out1 = out1.div(fnorm.expand_as(out1))
        fnorm = torch.norm(out2, p=2, dim=1, keepdim=True)
        out2 = out2.div(fnorm.expand_as(out2))
        
        dists = nn.PairwiseDistance()(out1, out2)
        loss = (labels) * torch.pow(dists, 2) + (1 - labels) * torch.pow(torch.clamp(self.margin - dists, min=0.0), 2)
        loss = torch.mean(loss)
        self.log("val_loss", loss, logger=True, on_epoch=True, prog_bar=True)   
        
        try:
            auc = roc_auc_score(labels.detach().cpu(), 1 - dists.detach().cpu())
        except:
            auc = 0
        self.log("val_auc", auc, logger=True, on_epoch=True, prog_bar=True)
        
    def train_dataloader(self):
        return train_loader

    def val_dataloader(self):
        return val_loader

In [15]:
model = Net()

In [16]:
import torchinfo

torchinfo.summary(model)

Layer (type:depth-idx)                   Param #
Net                                      --
├─Sequential: 1-1                        --
│    └─BatchNorm1d: 2-1                  1,920
│    └─Linear: 2-2                       245,760
│    └─ReLU: 2-3                         --
│    └─BatchNorm1d: 2-4                  512
│    └─Linear: 2-5                       65,536
│    └─BatchNorm1d: 2-6                  512
Total params: 314,240
Trainable params: 314,240
Non-trainable params: 0

In [17]:
trainer = pl.Trainer(
    accelerator='gpu', 
    devices=1,
    profiler='advanced',
    precision=32,
    check_val_every_n_epoch=1,
    max_epochs=10
)

trainer.fit(model)

Sanity Checking: 0it [00:00, ?it/s]

2023-05-21 13:11:43,072 [169980] WARNING  py.warnings:109: [JupyterRequire] C:\Users\Konder\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(

2023-05-21 13:11:43,775 [169980] WARNING  py.warnings:109: [JupyterRequire] C:\Users\Konder\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(



Training: 0it [00:00, ?it/s]

2023-05-21 13:11:43,829 [169980] WARNING  py.warnings:109: [JupyterRequire] C:\Users\Konder\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\trainer\connectors\logger_connector\result.py:229: UserWarning: You called `self.log('step', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(



Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [18]:
dists, labels = [], []
for x1, x2, target in tqdm(val_loader):
    out1 = model.forward(x1).detach().cpu()
    out2 = model.forward(x2).detach().cpu()
    
    fnorm = torch.norm(out1, p=2, dim=1, keepdim=True)
    out1 = out1.div(fnorm.expand_as(out1))
    fnorm = torch.norm(out2, p=2, dim=1, keepdim=True)
    out2 = out2.div(fnorm.expand_as(out2))
        
    dists.append(
        1 - nn.PairwiseDistance()(out1, out2).numpy()
    )
    labels.append(target.numpy())
dists = np.concatenate(dists)
labels = np.concatenate(labels)

  0%|          | 0/799 [00:00<?, ?it/s]

In [19]:
dists

array([0.80640864, 0.5427141 , 0.61773205, ..., 0.46092886, 0.699883  ,
       0.19064528], dtype=float32)

In [20]:
labels

array([0, 1, 0, ..., 1, 0, 1], dtype=int64)

In [21]:
roc_auc_score(labels, dists)

0.8702029210442249

In [22]:
train_dataset = ItemsDataset(train_pairs, train_data)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=args.batch_size,
    num_workers=0,
    drop_last=False,
    shuffle=False,
    pin_memory=True
)

In [23]:
features, labels = [], []
for x1, x2, target in tqdm(train_loader):
    out1 = model.forward(x1).detach().cpu()
    out2 = model.forward(x2).detach().cpu()
    
    fnorm = torch.norm(out1, p=2, dim=1, keepdim=True)
    out1 = out1.div(fnorm.expand_as(out1))
    fnorm = torch.norm(out2, p=2, dim=1, keepdim=True)
    out2 = out2.div(fnorm.expand_as(out2))
    
    features.append(np.concatenate([out1, out2, 1 - nn.PairwiseDistance()(out1, out2).numpy().reshape((-1, 1))], axis=1))
    labels.append(target.numpy())
features = np.concatenate(features)
labels = np.concatenate(labels)    

  0%|          | 0/1597 [00:00<?, ?it/s]

In [24]:
val_features, val_labels = [], []
for x1, x2, target in tqdm(val_loader):
    out1 = model.forward(x1).detach().cpu()
    out2 = model.forward(x2).detach().cpu()
    
    fnorm = torch.norm(out1, p=2, dim=1, keepdim=True)
    out1 = out1.div(fnorm.expand_as(out1))
    fnorm = torch.norm(out2, p=2, dim=1, keepdim=True)
    out2 = out2.div(fnorm.expand_as(out2))
    
    val_features.append(np.concatenate([out1, out2, 1 - nn.PairwiseDistance()(out1, out2).numpy().reshape((-1, 1))], axis=1))
    val_labels.append(target.numpy())
val_features = np.concatenate(val_features)
val_labels = np.concatenate(val_labels)

  0%|          | 0/799 [00:00<?, ?it/s]

In [25]:
from catboost import CatBoostClassifier, Pool, cv

2023-05-21 13:19:36,951 [169980] WARNING  py.warnings:109: [JupyterRequire] C:\Users\Konder\AppData\Roaming\Python\Python38\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index



In [26]:
train_pool = Pool(
    data=features,
    label=labels,
)

val_pool = Pool(
    data=val_features,
    label=val_labels,
)

In [27]:
params = {
    'loss_function': 'CrossEntropy',
    'custom_metric': ['AUC'],
    'task_type': 'CPU',
}

In [28]:
model_cb = CatBoostClassifier(**params, random_seed=56)
model_cb.fit(train_pool, eval_set=val_pool, verbose=False, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [29]:
np.max(model_cb.get_evals_result()['validation']['AUC'])

0.8732562904505206